In [ ]:
from functools import partial
import time
import json
import os
from glob import glob
import csv
import random
import urllib3
import itertools

import geopandas as gpd
import geojson
import folium
import requests
import requests_cache
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import shapely
from retry import retry
from tqdm.autonotebook import tqdm
import pyproj
import shapely
from functools import partial
from shapely.geometry import box, Polygon, MultiPolygon, GeometryCollection, shape
from shapely import wkt

In [ ]:
mpl.rcParams['figure.dpi'] = 200
pd.set_option('display.float_format', lambda x: f'{x:,.3f}')
pd.set_option("display.max_rows", 100)
urllib3.disable_warnings()
pd.set_option('display.width', 2000)
pd.set_option('max_colwidth', 400)

In [ ]:
def point_to_latlon(point):
    return point.y, point.x

def point_to_lonlat(point):
    return point.x, point.y

def point_to_xy(point):
    return point_to_lonlat(point)

def buffer_meters(geom, buffer):
    local_azimuthal_projection = f"+proj=aeqd +R=6371000 +units=m +lat_0={geom.centroid.y} +lon_0={geom.centroid.x}"

    wgs84_to_aeqd = partial(
        pyproj.transform,
        pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
        pyproj.Proj(local_azimuthal_projection),
    )

    aeqd_to_wgs84 = partial(
        pyproj.transform,
        pyproj.Proj(local_azimuthal_projection),
        pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
    )

    geom_transformed = shapely.ops.transform(wgs84_to_aeqd, geom)

    buffer = geom_transformed.buffer(buffer)

    buffer_wgs84 = shapely.ops.transform(aeqd_to_wgs84, buffer)
    
    return buffer_wgs84  

In [ ]:
df_city = gpd.read_file('../data/city_boundaries_100m.geojson')
df_city.sort_values('area_km2').tail(10)

In [ ]:
iloc = 0
poly = df_city.iloc[iloc].geometry


# Plot point and geom.
fig = folium.Figure(width=1000, height=500)
fm = folium.Map(
    tiles='OpenStreetMap',
    location=point_to_latlon(poly.centroid),
    zoom_start=10,
).add_to(fig)

folium.GeoJson(poly).add_to(fm)


fm

In [ ]:
# Adapted from https://github.com/JoaoCarabetta/osm-road-length

def simplify(geom):
    local_azimuthal_projection = f"+proj=aeqd +R=6371000 +units=m +lat_0={geom.centroid.y} +lon_0={geom.centroid.x}"

    wgs84_to_aeqd = partial(
        pyproj.transform,
        pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
        pyproj.Proj(local_azimuthal_projection),
    )

    aeqd_to_wgs84 = partial(
        pyproj.transform,
        pyproj.Proj(local_azimuthal_projection),
        pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
    )

    geom_transformed = shapely.ops.transform(wgs84_to_aeqd, geom)
    geom_simple = geom_transformed.simplify(500)
    geom_wgs84 = shapely.ops.transform(aeqd_to_wgs84, geom_simple)
    
    return geom_wgs84  




def to_geojson(x):

    if isinstance(x, shapely.geometry.multipolygon.MultiPolygon):
        x = max(x, key=lambda a: a.area)

    g = geojson.Feature(geometry=x, properties={}).geometry

    return g["coordinates"][0]


def swipe(x):
    return [[c[1], c[0]] for c in x]


def flatten(l):
    return [str(round(item, 4)) for sublist in l for item in sublist]


def to_overpass_coords(x):

    coords = to_geojson(x)
    coords = swipe(coords)
    coords = flatten(coords)
    coords = " ".join(coords)
    return coords

def threshold_func(g, value):

    return get_area(g) < value

def katana(geometry, threshold_func, threshold_value, count=0):
    """Split a Polygon into two parts across it's shortest dimension
    
    KUDOS https://snorfalorpagus.net/blog/2016/03/13/splitting-large-polygons-for-faster-intersections/
    """
    bounds = geometry.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    if threshold_func(geometry, threshold_value) or count == 250:
        # either the polygon is smaller than the threshold, or the maximum
        # number of recursions has been reached
        return [geometry]
    if height >= width:
        # split left to right
        a = box(bounds[0], bounds[1], bounds[2], bounds[1] + height / 2)
        b = box(bounds[0], bounds[1] + height / 2, bounds[2], bounds[3])
    else:
        # split top to bottom
        a = box(bounds[0], bounds[1], bounds[0] + width / 2, bounds[3])
        b = box(bounds[0] + width / 2, bounds[1], bounds[2], bounds[3])
    result = []
    for d in (
        a,
        b,
    ):
        c = geometry.intersection(d)
        if not isinstance(c, GeometryCollection):
            c = [c]
        for e in c:
            if isinstance(e, (Polygon, MultiPolygon)):
                result.extend(katana(e, threshold_func, threshold_value, count + 1))
    if count > 0:
        return result
    # convert multipart into singlepart
    final_result = []
    for g in result:
        if isinstance(g, MultiPolygon):
            final_result.extend(g)
        else:
            final_result.append(g)
    return final_result



def get_area(s):
    # s = shape(s)
    proj = partial(
        pyproj.transform, pyproj.Proj("epsg:4326"), pyproj.Proj("epsg:3857")
    )

    return shapely.ops.transform(proj, s).area / 1e6  # km


@retry(tries=2, delay=1, backoff=2, max_delay=60)
def overpass_request(geo, yomo=None, overpass_url='http://overpass-api.de/api/interpreter'):
    
    if yomo:
        date_str = f'[date:"{yomo}-01T00:00:00Z"]'
    else:
        date_str = ''
        
    overpass_query = f"""
        [out:json][timeout:25];
        (
          node["amenity"="hospital"]["emergency"="yes"](poly:"{geo}");
          way["amenity"="hospital"]["emergency"="yes"](poly:"{geo}");
          relation["amenity"="hospital"]["emercency"="yes"](poly:"{geo}");
        );
        out body;
        >;
        out skel qt;
    """
        

   
    
    response = requests.get(
        overpass_url,
        params={"data": overpass_query},
        proxies=proxies,
        verify=False,
        timeout=60,
    )
    
    return response


def get(geometry, yomo=None, overpass_url='http://overpass-api.de/api/interpreter', threshold_value=1000000/100):
    """Get Open Street Maps highways length in meters and object count for a given geometry
    It splits the regions to manage overpass turbo limits.
    For MultiPolygons, only the biggest polygon will be considered.
    Parameters
    ----------
    geometry : shapely.geometry
        A shapely polygon
    threshold_value : int, optional
        Maximum area in sq km to split the polygons, by default 1000000
    Returns
    -------
    pd.DataFrame
        Table indexed by highway with length sum in meters and observation count
    """

    geo = simplify(geometry)
    geoms = katana(geometry, threshold_func, threshold_value)
    responses = []
    print(len(geoms))
    for geo in geoms:
        geo = to_overpass_coords(geo)
        response = overpass_request(geo, yomo, overpass_url)
        responses.append([])
        
    
    return responses
    
    
# r = get(poly)

In [ ]:
ilocs = list(range(len(df_city)))
random.shuffle(ilocs)
        
backends = [
    'http://lz4.overpass-api.de/api/interpreter', 
    'http://z.overpass-api.de/api/interpreter',
    'https://overpass.openstreetmap.ru/api/interpreter',
    'https://overpass.nchc.org.tw/api/interpreter',
]
backends_r = itertools.cycle(backends)
        
    
for iloc in tqdm(ilocs):
    path = f'../data/hosp/{iloc}.csv'
    if os.path.exists(path):
        continue
    overpass_url = next(backends_r)
    poly = buffer_meters(df_city.iloc[iloc].geometry, 500)

    responses = get(poly, overpass_url=overpass_url)

    dfs = [pd.DataFrame.from_records(response.json()['elements']) for response in responses]
    df = pd.concat(dfs)
    df['city_iloc'] = iloc

    df.to_csv(path, index=False, quoting=csv.QUOTE_NONNUMERIC)
    time.sleep(10)
        

In [ ]:
# Merge into one df.
dfs = []
for iloc in tqdm(ilocs):
    path = f'../data/hosp/{iloc}.csv'
    if not os.path.exists(path):
        continue
        
    df = pd.read_csv(path)
    if len(df) == 0:
        continue
    dfs.append(df)
    
df_merged = pd.concat(dfs)
df_merged.to_csv('../data/osrm_hospital_location.csv')